In [1]:
import pandas as pd
import random
import os
import requests
import csv
import time

# --- Cấu hình Moodle API ---
MOODLE_URL = 'http://localhost:8100/webservice/rest/server.php'
TOKEN = 'ac6bc02c96ff699f2d0df05b9382e23b' # Thay thế bằng token Moodle của bạn
FORMAT = 'json'

# --- Hàm gọi Moodle API ---
def call_api(function, extra_params):
    """
    Gọi Moodle Web Service API.
    """
    params = {
        'wstoken': TOKEN,
        'moodlewsrestformat': FORMAT,
        'wsfunction': function
    }
    params.update(extra_params)
    try:
        response = requests.post(MOODLE_URL, data=params)
        response.raise_for_status() # Kiểm tra lỗi HTTP
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"❌ Lỗi khi gọi Moodle API '{function}': {e}")
        return {} # Trả về dictionary rỗng để tránh lỗi

# --- Hàm lấy Cluster của người dùng từ file CSV ---
def get_user_cluster(user_id):
    """
    Lấy thông tin cluster của người dùng từ file CSV.
    """
    file_path = "./synthetic_user_features_clustered.csv" # Đảm bảo đường dẫn đúng
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print("❌ Không tìm thấy file: synthetic_user_features_clustered.csv")
        return None
    user_row = df[df['userid'] == user_id]
    if user_row.empty:
        # print(f"⚠️ Không tìm thấy người dùng với ID: {user_id} trong file cluster.")
        return None # Trả về None nếu không tìm thấy cluster
    return user_row.iloc[0]['cluster']

# --- Hằng số và siêu tham số (Hyperparameters) cho Q-learning ---
LEARNING_RATE = 0.1    # Tốc độ học của thuật toán
DISCOUNT_FACTOR = 0.9  # Tầm quan trọng của phần thưởng trong tương lai
EXPLORATION_RATE = 0.2 # Tỷ lệ khám phá (chọn hành động ngẫu nhiên)

# Định nghĩa các hành động (Actions)
ACTIONS = [
    'read_new_resource',
    'review_old_resource',
    'attempt_new_quiz',
    'redo_failed_quiz',
    'skip_to_next_module'
]

# Ánh xạ loại hành động từ CSV sang các hành động đã định nghĩa
def get_action_from_type(activity_type):
    """
    Ánh xạ loại hoạt động từ log sang hành động Q-learning.
    Lưu ý: Cần logic phức tạp hơn để phân biệt attempt_new_quiz và redo_failed_quiz.
    """
    if activity_type == 'quiz':
        return 'attempt_new_quiz' # Giả định ban đầu là làm quiz mới
    elif activity_type == 'resource' or activity_type == 'hvp':
        return 'read_new_resource'
    else:
        return 'skip_to_next_module' # Hành động mặc định nếu không khớp

# Định nghĩa các khoảng rời rạc cho trạng thái (States)
# Điểm_trung_bình_quiz_module: 0-2 (Kém), 2-5 (Trung bình), 5-8 (Khá), 8-10 (Giỏi)
QUIZ_SCORE_BINS = [2, 5, 8, 10]
# Tỷ_lệ_đọc_tài_liệu: 0-40% (0), 41-80% (1), 81-100% (2)
READ_RATE_BINS = [0.4, 0.8, 1.0]

# Khởi tạo Q-table (sẽ được tải hoặc huấn luyện)
q_table = {}

# --- Hàm rời rạc hóa trạng thái ---
def discretize_state(avg_quiz_score, completion_rate, quiz_passed, cluster):
    """
    Chuyển đổi các giá trị liên tục/boolean của state thành các bin rời rạc.
    """
    # Rời rạc hóa điểm trung bình quiz
    quiz_bin = 0
    if avg_quiz_score > QUIZ_SCORE_BINS[2]: # > 8
        quiz_bin = 3
    elif avg_quiz_score > QUIZ_SCORE_BINS[1]: # > 5
        quiz_bin = 2
    elif avg_quiz_score > QUIZ_SCORE_BINS[0]: # > 2
        quiz_bin = 1

    # Rời rạc hóa tỷ lệ đọc tài liệu
    read_bin = 0
    if completion_rate > READ_RATE_BINS[1]: # > 0.8
        read_bin = 2
    elif completion_rate > READ_RATE_BINS[0]: # > 0.4
        read_bin = 1
    
    # quiz_passed và cluster đã ở dạng rời rạc (0, 1)
    # Đảm bảo cluster là số nguyên (hoặc 0 nếu None)
    cluster_val = int(cluster) if cluster is not None else 0
    return (quiz_bin, read_bin, int(quiz_passed), cluster_val)

# --- Hàm lấy giá trị Q-value ---
def get_q_value(state, action):
    """
    Lấy Q-value cho một cặp (state, action). Khởi tạo nếu chưa có.
    """
    if state not in q_table:
        q_table[state] = {a: 0.0 for a in ACTIONS}
    return q_table[state][action]

# --- Hàm cập nhật Q-table ---
def update_q_table(state, action, reward, next_state):
    """
    Cập nhật Q-value theo công thức Q-learning.
    """
    current_q = get_q_value(state, action)
    
    # Lấy Q-value tối đa của trạng thái tiếp theo
    if next_state not in q_table:
        max_future_q = 0.0
    else:
        max_future_q = max(q_table[next_state].values())

    # Công thức Q-learning
    new_q = current_q + LEARNING_RATE * (reward + DISCOUNT_FACTOR * max_future_q - current_q)
    q_table[state][action] = new_q

# --- Hàm tính toán phần thưởng (Reward) ---
def get_reward(current_state, action, next_state):
    """
    Tính toán phần thưởng dựa trên sự thay đổi trạng thái và hành động.
    """
    reward = 0
    # Phần thưởng khi quiz được pass (từ 0 -> 1)
    if current_state[2] == 0 and next_state[2] == 1:
        reward += 10
    
    # Phần thưởng khi completion rate tăng
    if next_state[1] > current_state[1]:
        reward += 5
    
    # Phần thưởng khi quiz score bin tăng
    # Điều này giúp khuyến khích cải thiện điểm số, ngay cả khi chỉ tăng 1 bin
    if next_state[0] > current_state[0]:
        reward += 7 
    
    # Hình phạt khi quiz score bin giảm
    if next_state[0] < current_state[0]:
        reward -= 5
    
    # Thêm phần thưởng/phạt cho các hành động cụ thể nếu cần
    # Ví dụ: nếu action là 'skip_to_next_module' nhưng điểm thấp -> phạt
    # if action == 'skip_to_next_module' and next_state[0] < 2: # Điểm kém
    #     reward -= 15
    
    return reward

# --- Hàm lưu Q-table vào file CSV ---
def save_q_table_to_csv(q_table, filename='q_table_results.csv'):
    """
    Lưu Q-table đã học vào file CSV.
    """
    q_table_data = []
    for state, actions_dict in q_table.items():
        for action, q_value in actions_dict.items():
            row = {
                'quiz_bin': state[0],
                'read_bin': state[1],
                'quiz_passed': state[2],
                'cluster': state[3],
                'action': action,
                'q_value': q_value
            }
            q_table_data.append(row)
    
    df = pd.DataFrame(q_table_data)
    df.to_csv(filename, index=False)
    print(f"✅ Q-table đã được lưu vào file '{filename}' thành công.")

# --- Hàm tải Q-table từ file CSV ---
def load_q_table_from_csv(filename='q_table_results.csv'):
    """
    Tải Q-table từ file CSV đã lưu.
    """
    q_table = {}
    if not os.path.exists(filename):
        print(f"❌ Không tìm thấy file '{filename}'. Vui lòng huấn luyện mô hình trước.")
        return {}
    
    df = pd.read_csv(filename)
    for _, row in df.iterrows():
        state = (row['quiz_bin'], row['read_bin'], row['quiz_passed'], row['cluster'])
        action = row['action']
        q_value = row['q_value']
        
        if state not in q_table:
            q_table[state] = {a: 0.0 for a in ACTIONS}
        
        q_table[state][action] = q_value
    print(f"✅ Q-table đã được tải từ file '{filename}' thành công.")
    return q_table

# --- Hàm huấn luyện mô hình từ dữ liệu lịch sử ---
def train_from_csv(file_path):
    """
    Huấn luyện Q-table từ dữ liệu lịch sử trong file CSV.
    """
    print("🚀 Bắt đầu huấn luyện mô hình Q-learning...")
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"❌ Không tìm thấy file dữ liệu lịch sử: {file_path}. Không thể huấn luyện.")
        return

    df = df.sort_values(by=['userid', 'time'])

    for userid in df['userid'].unique():
        user_data = df[df['userid'] == userid].reset_index(drop=True)
        for i in range(len(user_data) - 1):
            current_row = user_data.iloc[i]
            next_row = user_data.iloc[i+1]
            
            # Đảm bảo các dòng này thuộc cùng một module để so sánh trạng thái
            if current_row['sectionid'] != next_row['sectionid']:
                continue

            state = discretize_state(
                current_row['avg_quiz_score'],
                current_row['completion_rate'],
                current_row['quiz_passed'],
                current_row['cluster']
            )
            
            action = get_action_from_type(next_row['type'])
            
            next_state = discretize_state(
                next_row['avg_quiz_score'],
                next_row['completion_rate'],
                next_row['quiz_passed'],
                next_row['cluster']
            )

            reward = get_reward(state, action, next_state)
            update_q_table(state, action, reward, next_state)
    print("✅ Huấn luyện Q-learning hoàn tất.")

# --- Hàm gợi ý hành động tốt nhất ---
def suggest_next_action(current_state, q_table):
    """
    Gợi ý hành động tốt nhất dựa trên trạng thái hiện tại và Q-table.
    """
    if current_state not in q_table:
        # Nếu trạng thái chưa từng được học, chọn hành động ngẫu nhiên
        return random.choice(ACTIONS), 0.0
    
    q_values = q_table[current_state]
    best_action = max(q_values, key=q_values.get)
    best_q_value = q_values[best_action]
    
    return best_action, best_q_value

# --- Hàm chính để theo dõi log và đưa ra gợi ý ---
def track_and_suggest_new_rows(log_file_path, trained_q_table, poll_interval=2):
    """
    Theo dõi file log mới, tính toán trạng thái và đưa ra gợi ý hành động.
    """
    print(f"\n🔍 Đang theo dõi file log: {log_file_path} và gợi ý hành động...")
    seen_lines = 0
    output_path = './user_insight.csv' # File để ghi các insight đã tính toán

    # Tạo file output nếu chưa có
    if not os.path.exists(output_path):
        with open(output_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                'userid', 'courseid', 'sectionid', 'type', 
                'avg_quiz_score', 'completion_rate', 'quiz_passed', 
                'cluster', 'time'
            ])

    while True:
        try:
            # Đọc file log mới nhất
            df_log = pd.read_csv(log_file_path)
            current_lines = len(df_log)

            if current_lines > seen_lines:
                new_rows = df_log.iloc[seen_lines:]
                print(f"\n🆕 Phát hiện {len(new_rows)} dòng log mới:")
                seen_lines = current_lines

                for _, row in new_rows.iterrows():
                    userid = int(row['userid'])
                    courseid = int(row['courseid'])
                    sectionid = int(row['sectionid'])
                    typ = str(row['type'])
                    timestamp = int(row['time'])

                    print(f"\n--- Xử lý hành động mới của User {userid} ({typ}) tại Section {sectionid} ---")

                    # Gọi API Moodle để lấy dữ liệu thời gian thực
                    avg_quiz_score_data = call_api('local_userlog_get_avg_quiz_score', {
                        'userid': userid, 'courseid': courseid
                    })
                    avg_quiz_score = avg_quiz_score_data.get('avg_quiz_score', 0.0)

                    total_resource_data = call_api('local_userlog_get_total_resources_by_section', {
                        'sectionid': sectionid,
                        'objecttypes[0]': 'resource',
                        'objecttypes[1]': 'hvp',
                    })
                    total_resource = total_resource_data.get('total_resources', 0)

                    viewed_resource_data = call_api('local_userlog_get_viewed_resources_distinct_by_section', {
                        'userid': userid,
                        'courseid': courseid,
                        'sectionid': sectionid,
                        'objecttypes[0]': 'resource',
                        'objecttypes[1]': 'hvp',
                    })
                    viewed_resource = viewed_resource_data.get('viewed_resources', 0)

                    completion_rate = viewed_resource / total_resource if total_resource > 0 else 0

                    quiz_passed_data = call_api('local_userlog_get_latest_quiz_pass_status_by_section', {
                        'userid': userid,
                        'sectionid': sectionid,
                    })
                    quiz_passed = quiz_passed_data.get('is_passed', 0)

                    cluster = get_user_cluster(userid)

                    # Ghi insight vào file user_insight.csv
                    with open(output_path, 'a', newline='') as f:
                        writer = csv.writer(f)
                        writer.writerow([
                            userid,
                            courseid,
                            sectionid,
                            typ,
                            avg_quiz_score,
                            round(completion_rate, 2),
                            int(quiz_passed),
                            cluster,
                            timestamp
                        ])
                    
                    # Tính toán trạng thái hiện tại của người dùng
                    current_state = discretize_state(
                        avg_quiz_score,
                        completion_rate,
                        quiz_passed,
                        cluster
                    )

                    # Gợi ý hành động tốt nhất
                    best_action, q_value = suggest_next_action(current_state, trained_q_table)
                    print(f"👉 Trạng thái hiện tại của User {userid}: {current_state}")
                    print(f"💡 Hệ thống đề xuất hành động: '{best_action}' (Q-value = {q_value:.2f})")
            
            time.sleep(poll_interval)

        except FileNotFoundError:
            print("⚠️ File log chưa tồn tại. Đợi...")
            time.sleep(poll_interval)
        except pd.errors.EmptyDataError:
            print("⚠️ File log rỗng. Đợi dữ liệu mới...")
            time.sleep(poll_interval)
        except Exception as e:
            print(f"❌ Lỗi trong quá trình theo dõi và gợi ý: {e}")
            # Có thể thêm logic để thoát hoặc tiếp tục tùy theo loại lỗi
            time.sleep(poll_interval) # Đợi trước khi thử lại

# --- Quy trình chạy chính ---
if __name__ == "__main__":
    # Bước 1: Huấn luyện mô hình từ dữ liệu lịch sử (chỉ chạy khi cần)
    # Nếu bạn đã có file user_insight.csv được tạo ra từ lần chạy trước,
    # bạn có thể bỏ qua bước này và chỉ tải Q-table.
    # Tuy nhiên, để đảm bảo Q-table được cập nhật với dữ liệu mới nhất,
    # bạn nên chạy lại bước này định kỳ.
    train_from_csv('user_insight.csv') # File này được tạo ra từ track_new_rows

    # Bước 2: Tải Q-table đã huấn luyện để sử dụng
    # Nếu không có file q_table_results.csv, hàm này sẽ trả về dictionary rỗng.
    trained_q_table = load_q_table_from_csv('q_table_results.csv')

    # Bước 3: Bắt đầu theo dõi file log và đưa ra gợi ý thời gian thực
    # Thay đổi đường dẫn tới file log thực tế của Moodle
    log_file_path = "/Users/nguyenhuuloc/Documents/MyComputer/moodledata/local_userlog_data/user_log_summary.csv"
    track_and_suggest_new_rows(log_file_path, trained_q_table)



🚀 Bắt đầu huấn luyện mô hình Q-learning...
✅ Huấn luyện Q-learning hoàn tất.
✅ Q-table đã được tải từ file 'q_table_results.csv' thành công.

🔍 Đang theo dõi file log: /Users/nguyenhuuloc/Documents/MyComputer/moodledata/local_userlog_data/user_log_summary.csv và gợi ý hành động...

🆕 Phát hiện 20 dòng log mới:

--- Xử lý hành động mới của User 4 (quiz) tại Section 42 ---
👉 Trạng thái hiện tại của User 4: (0, 2, 1, 0)
💡 Hệ thống đề xuất hành động: 'attempt_new_quiz' (Q-value = 0.25)

--- Xử lý hành động mới của User 4 (resource) tại Section 38 ---
👉 Trạng thái hiện tại của User 4: (0, 2, 0, 0)
💡 Hệ thống đề xuất hành động: 'read_new_resource' (Q-value = 0.13)

--- Xử lý hành động mới của User 4 (hvp) tại Section 38 ---
👉 Trạng thái hiện tại của User 4: (0, 2, 0, 0)
💡 Hệ thống đề xuất hành động: 'read_new_resource' (Q-value = 0.13)

--- Xử lý hành động mới của User 4 (quiz) tại Section 42 ---
👉 Trạng thái hiện tại của User 4: (0, 2, 1, 0)
💡 Hệ thống đề xuất hành động: 'attempt_new_quiz' 

KeyboardInterrupt: 